In [ ]:
import jax.numpy as np
import matplotlib.pyplot as plt

from ode_filters.filters import ekf1_sqr_loop, rts_sqr_smoother_loop
from ode_filters.measurement import ODEInformation
from ode_filters.priors import IWP, taylor_mode_initialization

## Example Problem: Logistic ODE

$$\dot{x}(t) = x(t)(1-x(t)), \quad x(0) = x_0 = 0.01, \quad t\in [0,10]$$


### 1. Define the Initial Value Problem (IVP)


In [ ]:
def vf(x, *, t):
    return x * (1 - x)


x0 = np.array([0.01])
tspan = [0, 10]
d = x0.shape[0]

### 2. Select a Prior

e.g. a q-times integrated wiener process:


In [ ]:
q = 2
xi = 0.5 * np.eye(d)
prior = IWP(q, d, Xi=xi)
mu_0, Sigma_0_sqr = taylor_mode_initialization(vf, x0, q)

### 4. Define the measuement model

e.g. the naive ODE information operator


In [ ]:
measure = ODEInformation(vf, prior.E0, prior.E1)

### 5. Apply ODE filtering and smoothing loop

e.g. EKF1 filtering and RTS smoothing.


In [ ]:
N = 21

(
    m_seq,
    P_seq_sqr,
    m_pred_seq,
    P_pred_seq_sqr,
    G_back_seq,
    d_back_seq,
    P_back_seq_sqr,
    mz_seq,
    Pz_seq_sqr,
    log_likelihood,
) = ekf1_sqr_loop(mu_0, Sigma_0_sqr, prior, measure, tspan, N)


In [ ]:
m_smoothed, P_smoothed_sqr = rts_sqr_smoother_loop(
    m_seq[-1], P_seq_sqr[-1], G_back_seq, d_back_seq, P_back_seq_sqr, N
)

### 6. Visualise the results


In [ ]:
m_seq = np.array(m_seq)
P_seq_sqr = np.array(P_seq_sqr)


In [ ]:
ts = np.linspace(tspan[0], tspan[1], N + 1)
plt.figure(figsize=(10, 4), dpi=600)
plt.xlabel("t")
plt.ylabel("x(t)")
plt.plot(ts, m_seq[:, 0], label="filtered estimate")
P_seq = np.matmul(np.transpose(P_seq_sqr, (0, 2, 1)), P_seq_sqr)
margin = 2 * np.sqrt(P_seq[:, 0, 0])
plt.fill_between(
    ts,
    m_seq[:, 0] - margin,
    m_seq[:, 0] + margin,
    alpha=0.5,
    label=r"2 $\sigma$ interval",
)
plt.legend()
plt.show()

### 7. JIT compilation

The filter loop is compatible with `jax.jit`. Since `prior`, `measure`, and `N` are not JAX arrays, they must be marked as static arguments so the tracer treats them as compile-time constants.


In [ ]:
import jax

# JIT-compile the filter loop.
# prior (arg 2), measure (arg 3), and N (arg 5) are not JAX arrays,
# so they must be static.
jit_filter = jax.jit(ekf1_sqr_loop, static_argnums=(2, 3, 5))

# First call traces and compiles; subsequent calls with the same
# static args reuse the compiled kernel.
filter_result = jit_filter(mu_0, Sigma_0_sqr, prior, measure, tspan, N)
log_likelihood_jit = filter_result[-1]
print(f"Log-likelihood (JIT): {log_likelihood_jit:.6f}")


Since the filter is now traceable, we can differentiate through it. A typical use case is tuning the diffusion scale $\xi$ by maximising the log marginal likelihood:


In [ ]:
def neg_log_likelihood(xi_scalar):
    """Negative log marginal likelihood as a function of diffusion scale."""
    xi_matrix = xi_scalar * np.eye(d)
    prior_opt = IWP(q, d, Xi=xi_matrix)
    measure_opt = ODEInformation(vf, prior_opt.E0, prior_opt.E1)
    *_, ll = ekf1_sqr_loop(mu_0, Sigma_0_sqr, prior_opt, measure_opt, tspan, N)
    return -ll


# JIT-compile the gradient of the loss
grad_nll = jax.jit(jax.grad(neg_log_likelihood))

xi_val = 0.5
print(f"xi = {xi_val}")
print(f"  neg log-likelihood : {neg_log_likelihood(xi_val):.6f}")
print(f"  gradient d(-ll)/dxi: {grad_nll(xi_val):.6f}")

In [ ]:
# The filter and smoother can be combined into a single JIT-compiled function.
# The filter returns Python lists of traced values, so we stack them with
# np.array() before passing to the smoother, which expects indexed arrays.
def filter_and_smooth(mu_0, Sigma_0_sqr, tspan):
    (
        m_seq,
        P_seq_sqr,
        _,
        _,
        G_back_seq,
        d_back_seq,
        P_back_seq_sqr,
        _,
        _,
        log_lik,
    ) = ekf1_sqr_loop(mu_0, Sigma_0_sqr, prior, measure, tspan, N)

    m_smooth, P_smooth_sqr = rts_sqr_smoother_loop(
        m_seq[-1],
        P_seq_sqr[-1],
        np.array(G_back_seq),
        np.array(d_back_seq),
        np.array(P_back_seq_sqr),
        N,
    )
    return m_smooth, P_smooth_sqr, log_lik


jit_filter_and_smooth = jax.jit(filter_and_smooth)
m_smooth_jit, P_smooth_sqr_jit, log_lik_jit = jit_filter_and_smooth(
    mu_0, Sigma_0_sqr, tspan
)
print(f"Smoothed mean shape : {m_smooth_jit.shape}")
print(f"Log-likelihood (JIT): {log_lik_jit:.6f}")